In [1]:
import pycircos
import warnings
import patchworklib as pw 
import matplotlib.pyplot as plt

from os.path import isdir, isfile
from PyCircos_Utils import *
from PyPDF2 import PdfMerger
from os import listdir, mkdir

warnings.filterwarnings("ignore")

rcParams = {'font.size': 20, 'font.weight': 'normal', 'font.family': 'sans-serif',
            'axes.unicode_minus':False, 'axes.labelweight':'normal'}

plt.rcParams.update(rcParams)

Garc    = pycircos.Garc
Gcircle = pycircos.Gcircle

osa_len, osb_len = 2932766, 3046682

/Users/harihara/anaconda3/lib/python3.7/site-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [ ]:
novel_contigs = {}
novel_contig_path = '/Users/harihara/Mount-2/hotspring_metagenome/Synechococcus_paper_analysis/\
Ref_Guided_Scaffolding_Clustering_Aug_2022/Missing_Contig_Coords_Aug/'
samples = listdir(novel_contig_path+'OSA/')
for s in samples:
    if(s.startswith("Hot") and s.endswith(".txt")):
        df_osa = pd.read_csv(novel_contig_path+'OSA/'+s, sep = "\t")
        df_osa.loc[(df_osa['Start'] < 0), 'Start'] += osa_len
        df_osa.loc[(df_osa['End'] < 0), 'End'] += osa_len
        df_osa_grp = df_osa.sort_values(by = ['Contig','Start']).groupby(['Contig']).apply(Max_Clique_Interval_Graph)
    
        df_osb = pd.read_csv(novel_contig_path+'OSB/'+s, sep = "\t")
        df_osb.loc[(df_osb['Start'] < 0), 'Start'] += osb_len
        df_osb.loc[(df_osb['End'] < 0), 'End'] += osb_len
        df_osb_grp = df_osb.sort_values(by = ['Contig','Start']).groupby(['Contig']).apply(Max_Clique_Interval_Graph)
    
        novel_contigs[s.replace(".txt","")] = {'OSA':df_osa_grp,'OSB':df_osb_grp}

In [ ]:
grp_path = '/Users/harihara/Mount-2/hotspring_metagenome/Synechococcus_paper_analysis/\
Ref_Guided_Scaffolding_Clustering_Aug_2022/BLAST_All_vs_All/Containment_Clusters_Filtered_90_75_1000.txt'
df_novel_filtered = pd.read_csv(grp_path, sep = "\t")
d = df_novel_filtered.groupby('GroupID')['Contig'].apply(list).to_dict()

out_dir = '/Users/harihara/Research-Activities/Plots/Hot_Spring_Plots/Synechococcus-Paper/Novel_Groups_Circos_Aug_2022/'
if not isdir(out_dir):
    mkdir(out_dir)


In [ ]:
i = 0
for g in list(d.keys()):
    if isfile(out_dir+g+'.pdf'):
        continue
        
    contigs = d[g]
    osa_contig_count, osb_contig_count = 0, 0
    osa_contigs, osb_contigs = [], []
    df_osa, df_osb = pd.DataFrame(), pd.DataFrame()
    
    for c in contigs:
        splits = c.split('_')
        
        if len(splits) == 5:
            sample = splits[0]+'_'+splits[1]
            genome = splits[2].upper()
            contig = splits[3]+'_'+splits[4]
        if len(splits) == 4:
            sample = splits[0]
            genome = splits[1].upper()
            contig = splits[2]+'_'+splits[3]
        if genome == "OSA":
            osa_contig_count += 1
        if genome == "OSB":
            osb_contig_count += 1
        
        try:
            row = novel_contigs[sample][genome].loc[contig]
            row['Group'] = g
            row['Sample'] = sample
            if genome == 'OSA':
                df_osa = df_osa.append(row)
            elif genome == 'OSB':
                df_osb = df_osb.append(row)
        except:
            print("Missing...",sample,genome,contig)
            pass
    
    if len(df_osa) > 0:
        df_osa = df_osa.reset_index()
    if len(df_osb) > 0:
        df_osb = df_osb.reset_index()
    
    osa_counts, osb_counts, osa_gbl_counts, osb_gbl_counts = Make_Counts(df_osa, df_osb, osa_len, osb_len)
    
    garc_osa = Garc(arc_id="OSA", interspace=0, linewidth=0, facecolor="#FFFFFF00", raxis_range=(0,10),
                    label=("Synechococcus Sub.Sp A\n"+"#Contigs (in Group):"+str(osa_contig_count)+
                           "\n#Contigs (in Graph):"+str(len(df_osa))), labelsize = 24,label_visible=True)
    garc_osb = Garc(arc_id="OSB", interspace=0, linewidth=0, facecolor="#FFFFFF00", raxis_range=(0,10),
                    label=("Synechococcus Sub.Sp B\n"+"#Contigs (in Group):"+str(osb_contig_count)+
                           "\n#Contigs (in Graph):"+str(len(df_osb))), labelsize = 24,label_visible=True)
    
    gcircle_osa = Gcircle(fig=pw.Brick._figure)
    gcircle_osa.add_garc(garc_osa)
    gcircle_osa.set_garcs()
    ylim = int(max(np.max(osa_gbl_counts), np.max(osb_gbl_counts)))+0.000001
    
    gcircle_osa.lineplot('OSA',osa_gbl_counts+0.000001,  raxis_range=(800,1000),
                         rlim = (0, ylim), linewidth = 4, linecolor = 'green')
    
    gcircle_osb = Gcircle(fig=pw.Brick._figure)
    gcircle_osb.add_garc(garc_osb)
    gcircle_osb.set_garcs()
    gcircle_osb.lineplot('OSB',osb_gbl_counts+0.000001,  raxis_range=(800,1000),
                         rlim = (0, ylim), linewidth = 4, linecolor = 'orange')
    
    pw.param["margin"] = 0.0001
    
    circos12 = pw.cBrick(ax=gcircle_osa.ax, figsize = (16,10)) | pw.cBrick(ax=gcircle_osb.ax, figsize = (16,10))
    circos12.set_suptitle(g, size = 60)
    circos12.savefig(out_dir+g+'.pdf')
    
    print(i, g)

    i+=1
    plt.close("all")

In [4]:
import resource
soft, hard = resource.getrlimit(resource.RLIMIT_NOFILE)
resource.setrlimit(resource.RLIMIT_NOFILE, (hard, hard))

out_dir = '/Users/harihara/Research-Activities/Plots/Hot_Spring_Plots/Synechococcus-Paper/Novel_Groups_Circos_Aug_2022/'
pdfs = listdir(out_dir)
pdfs.sort()
merger = PdfMerger()
for pdf in pdfs:
    if pdf.startswith('Group'):
        merger.append(out_dir+'/'+pdf)
        print(pdf)

Group_0.pdf
Group_1.pdf
Group_10.pdf
Group_100.pdf
Group_101.pdf
Group_108.pdf
Group_110.pdf
Group_112.pdf
Group_115.pdf
Group_118.pdf
Group_119.pdf
Group_120.pdf
Group_121.pdf
Group_122.pdf
Group_123.pdf
Group_127.pdf
Group_129.pdf
Group_130.pdf
Group_131.pdf
Group_132.pdf
Group_135.pdf
Group_138.pdf
Group_139.pdf
Group_14.pdf
Group_140.pdf
Group_15.pdf
Group_152.pdf
Group_155.pdf
Group_160.pdf
Group_162.pdf
Group_17.pdf
Group_173.pdf
Group_174.pdf
Group_178.pdf
Group_179.pdf
Group_18.pdf
Group_182.pdf
Group_189.pdf
Group_191.pdf
Group_193.pdf
Group_197.pdf
Group_198.pdf
Group_199.pdf
Group_201.pdf
Group_204.pdf
Group_205.pdf
Group_208.pdf
Group_21.pdf
Group_212.pdf
Group_222.pdf
Group_223.pdf
Group_226.pdf
Group_227.pdf
Group_229.pdf
Group_235.pdf
Group_237.pdf
Group_244.pdf
Group_249.pdf
Group_254.pdf
Group_256.pdf
Group_257.pdf
Group_258.pdf
Group_259.pdf
Group_266.pdf
Group_267.pdf
Group_268.pdf
Group_269.pdf
Group_27.pdf
Group_270.pdf
Group_273.pdf
Group_279.pdf
Group_282.pdf
Gro

In [5]:
merger.write(out_dir+'/Circos.pdf')
merger.close()

In [ ]:
osa_counts